In [104]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from tqdm.notebook import tqdm
import time
import random
import requests
import io


In [105]:
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", 20)

In [106]:
def get_sp500_tickers():
    print("Fetching S&P 500 tickers from Wikipedia...")

    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/91.0.4472.124 Safari/537.36"
        )
    }

    r = requests.get(url, headers=headers, timeout=10)
    r.raise_for_status()

    tables = pd.read_html(io.StringIO(r.text))
    tickers = tables[0]["Symbol"].astype(str).str.strip().unique().tolist()

    # Hard cap at 500
    if len(tickers) > 800:
        tickers = tickers[:500]

    print(f"Successfully loaded {len(tickers)} tickers from Wikipedia.")
    return tickers


tickers = get_sp500_tickers()
print(f"Total tickers we extracted : {len(tickers)}")

Fetching S&P 500 tickers from Wikipedia...
Successfully loaded 503 tickers from Wikipedia.
Total tickers we extracted : 503


In [107]:
def get_financials(ticker, max_years=3):
    time.sleep(random.uniform(0.1, 0.3))

    try:
        t = yf.Ticker(ticker)
        info = t.info
        fin = t.financials
        bal = t.balance_sheet
        cash = t.cashflow
    except Exception:
        return None

    if fin is None or fin.empty:
        return None

    # Only real year columns (ignore TTM or weird)
    valid_cols = [c for c in fin.columns if hasattr(c, "year")]
    valid_cols = sorted(valid_cols, reverse=True)[:max_years]

    if not valid_cols:
        return None

    def safe(df, row, col):
        try:
            return df.loc[row, col]
        except Exception:
            return np.nan

    rows = []
    currency = info.get("currency", "Unknown")

    # possible label variants for total equity in Yahoo balance sheet||not working
    equity_labels = [
        "Total Stockholder Equity",
        "Stockholders Equity",
        "Total Equity",
        "Common Stock Equity",
        "Shareholders Equity"
    ]

    for col in valid_cols:
        year = col.year

        revenue = safe(fin, "Total Revenue", col)
        net_income = safe(fin, "Net Income", col)

        ebitda_val = safe(fin, "EBITDA", col)
        if pd.isna(ebitda_val):
            ebitda_val = safe(fin, "Ebitda", col)

        gross_profit = safe(fin, "Gross Profit", col)

        total_assets = safe(bal, "Total Assets", col)
        total_liabilities = safe(bal, "Total Liab", col)
        if pd.isna(total_liabilities):
            total_liabilities = safe(
                bal, "Total Liabilities Net Minority Interest", col
            )

        total_equity = np.nan
        for label in equity_labels:
            val = safe(bal, label, col)
            if pd.notna(val):
                total_equity = val
                break

        operating_cashflow = safe(
            cash, "Total Cash From Operating Activities", col
        )

        rows.append({
            "ticker": ticker,
            "company_name": info.get("longName", ticker),
            "country": info.get("country", "Unknown"),
            "industry": info.get("industry", "Unknown"),
            "year": year,
            "revenue": revenue,
            "revenue_unit": currency,
            "net_income": net_income,
            "gross_profit": gross_profit,
            "ebitda": ebitda_val,
            "total_assets": total_assets,
            "total_liabilities": total_liabilities,
            "employees": info.get("fullTimeEmployees"),
            "market_cap": info.get("marketCap")
        })

    return rows





In [108]:
main_tickers = tickers[:]  # fetched earlier

# Optional manual backup loader (NOT  a fallback)
backup_tickers = []
backup_url = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv"

try:
    backup_df = pd.read_csv(backup_url)
    backup_tickers = backup_df["Symbol"].astype(str).str.strip().unique().tolist()
    print(f"Backup GitHub tickers loaded: {len(backup_tickers)}")
except:
    print("Backup source not available.")

target_tickers = main_tickers[:499]
print(f"Using {len(target_tickers)} tickers for extraction")


Backup GitHub tickers loaded: 503
Using 499 tickers for extraction


In [109]:
from tqdm.notebook import tqdm

data = []
failed = []

print("Starting extraction...")
for t in tqdm(target_tickers):
    result = get_financials(t)
    if result:
        data.extend(result)
    else:
        failed.append(t)

print("Finished extraction.")
print("Successful rows:", len(data))
print("Failed tickers:", len(failed))


Starting extraction...


  0%|          | 0/499 [00:00<?, ?it/s]

Exception ignored from cffi callback <function buffer_callback at 0x7e205d54c4a0>:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/curl_cffi/curl.py", line 100, in buffer_callback
    @ffi.def_extern()
    
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(data)
print("Raw dataset shape:", df.shape)

df = df.dropna(subset=["revenue"])
df = df[df["revenue_unit"] == "USD"]
df = df[df.groupby("ticker")["year"].transform("count") >= 2]

print("Final companies:", df["ticker"].nunique())
print("Final rows:", len(df))
df.head()


**NULL Value Data **

In [ ]:
print("\nMissing values per column:")
display(df.isna().sum())






In [ ]:
print("Missing values percentage per column:")
(df.isna().mean() * 100).round(2).sort_values(ascending=False)

In [ ]:
kpi_cols = [
    "revenue", "net_income", "gross_profit", "ebitda", "total_assets",
    "total_liabilities", "total_equity", "operating_cashflow",
    "employees", "market_cap"
]

print("Missing values among key KPIs:")
df[kpi_cols].isna().sum().sort_values(ascending=False)




In [ ]:
coverage = (df.notna().mean() * 100).round(2)
coverage.sort_values().plot(kind="barh", figsize=(10,8), title="Column Data Availability (%)")
plt.xlabel("Percent Non-null")
plt.tight_layout()
plt.show()

In [ ]:
print("Companies with highest missing values:")
df.groupby("ticker").apply(lambda x: x.isna().sum().sum()).sort_values(ascending=False).head(20)


In [ ]:
import seaborn as sns

null_percent = (df.isna().mean() * 100).round(2)

fig, ax = plt.subplots(figsize=(10,6))
sns.barplot(x=null_percent.index, y=null_percent.values)
plt.xticks(rotation=45, ha='right')
plt.title("Missing Values by Column (%)")
plt.ylabel("Percentage Missing")
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(14,6))
sns.heatmap(df.isna(), cbar=False)
plt.title("Missing Data Heatmap by Row")
plt.show()


KPI Engineering

In [ ]:
df = df.sort_values(["ticker", "year"])
df["enterprise_value"] = df["market_cap"] + df["total_liabilities"] - df["total_assets"]
df["ev_to_ebitda"] = df["enterprise_value"] / df["ebitda"]
df["gross_margin"] = df["gross_profit"] / df["revenue"]
df["net_profit_margin"] = df["net_income"] / df["revenue"]
df["operating_margin"] = df["ebitda"] / df["revenue"]
df["roa"] = df["net_income"] / df["total_assets"]
df["revenue_growth"] = df.groupby("ticker")["revenue"].pct_change()
df["net_income_growth"] = df.groupby("ticker")["net_income"].pct_change()
df["revenue_per_employee"] = df["revenue"] / df["employees"]




In [ ]:
empty_columns = df.columns[df.isnull().all()].tolist()
if empty_columns:
    print(f"The following columns are completely empty: {empty_columns}")
else:
    print("No columns are completely empty in the DataFrame.")

Visuals/Plots

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker



# Prepare latest year slice
latest_year = df["year"].max()
latest = df[df["year"] == latest_year].copy()
print("Latest year:", latest_year, "| Rows:", len(latest))

# Cleaning  float columns and removing invalid values
latest = latest.replace([np.inf, -np.inf], np.nan)


def fmt(x, pos):
    if x >= 1e9: return "${:1.1f}B".format(x * 1e-9)
    if x >= 1e6: return "${:1.0f}M".format(x * 1e-6)
    return str(x)

top10 = latest.sort_values("revenue", ascending=False).head(10)

fig, ax = plt.subplots()
ax.bar(top10["company_name"], top10["revenue"], color="#1f77b4")
ax.yaxis.set_major_formatter(mticker.FuncFormatter(fmt))
plt.xticks(rotation=45, ha="right")
plt.title(f"Top 10 Companies by Revenue (USD) - {latest_year}")
plt.tight_layout()
plt.show()



In [ ]:
top_rpe = latest.sort_values("revenue_per_employee", ascending=False).head(10)

fig, ax = plt.subplots()
ax.bar(top_rpe["company_name"], top_rpe["revenue_per_employee"])
ax.yaxis.set_major_formatter(mticker.FuncFormatter(fmt))
plt.xticks(rotation=45, ha="right")
plt.title(f"Top 10 Revenue per Employee ({latest_year})")
plt.tight_layout()
plt.show()


In [ ]:
ev_group = latest.groupby("industry")["ev_to_ebitda"].median().sort_values().head(15)

fig, ax = plt.subplots()
ev_group.plot(kind="bar")
plt.xticks(rotation=45, ha="right")
plt.ylabel("Median EV/EBITDA")
plt.title("Industry Valuation Comparison (EV/EBITDA)")
plt.tight_layout()
plt.show()


In [ ]:
df.to_csv("financial_data_structured.csv", index=False)
print("Saved → financial_data_structured.csv")


📌 Key Insights from the Dataset

*   The dataset contains 278 publicly traded U.S. companies with ~3 years of historical financials, fulfilling the assignment requirement of 100–500 companies.

* Revenue, Net Income, EBITDA, and Asset values are consistently reported, enabling reliable profitability and efficiency analysis


*   Revenue distribution is highly concentrated — a small number of firms (e.g., mega-cap companies) dominate revenue scale, observable in the Top 10 Revenue visualization



*   Profitability varies significantly across industries — companies with relatively lower revenues can still achieve high net profit margins, highlighting operational efficiency differences.

*   Revenue Growth and Net Income Growth show substantial variability, reflecting differences in post-pandemic recovery trajectories across industries.

*   Productivity metrics such as Revenue per Employee offer strong insight into workforce efficiency and competitive positioning
*   ROA (Return on Assets) and margin metrics provide meaningful performance comparison without requiring unavailable balance sheet elements (e.g., equity).
Engineered KPIs enable deeper financial analysis beyond raw financial reporting (margins, efficiency, valuation multiples, growth).




⚠️ Limitations & Data Quality Constraints



*   Yahoo Finance provides inconsistent field availability — key balance sheet fields such as total_equity and operating_cashflow were missing for almost all companies.

*   Derived KPIs reliant on unavailable data such as ROE, Debt-to-Equity, and Cash Conversion were removed to preserve analytical integrity
*   Some KPIs (e.g., Growth metrics) inherently exhibit fewer values due to mathematical dependency on previous years, resulting in expected missingness for year-1.

*   Revenue Growth is influenced by macroeconomic and sector volatility, so interpretation depends heavily on external context beyond financial statements.

*   EBITDA and Gross Profit are missing in ~10–12% of records depending on sector reporting policies.
*   Market Cap and Enterprise Value may fluctuate daily; extracted values represent a point-in-time snapshot rather than averaged valuations.





.




